In [2]:
from DbConnector import DbConnector
#from part2 import Database
from tabulate import tabulate

# Set up the program
from dotenv import load_dotenv
import os
import numpy as np
from datetime import datetime, timedelta
from haversine import haversine, Unit
from pprint import pprint

load_dotenv()

connector = DbConnector()

You are connected to the database: assignment3
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

### Task 2
Find the average number of activities per user.

In [58]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [

        {"$group": {"_id":  "$user_id", "count": {"$sum": 1}}},
        {
        "$group": {
            "_id": None,
            "average avtivites": {"$avg": "$count"}
            }
        },
        {
            "$project": {
                "_id": 0,
                "average_activities": 0
            }
        }

    ]
)

for row in rows:
    print(row)






{'average avtivites': 51.82236842105263}


### Task 3

Find the top 20 users with the highest number of activities.

In [15]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$group": {"_id":  "$user_id", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {
            "$setWindowFields": {
                "sortBy": {"count": -1},
                "output": {"Top": {"$rank": {}}},
            }
        },
        {"$limit": 20},
    ]
)

for row in rows:
    pprint(row)

{'Top': 1, '_id': '025', 'count': 715}
{'Top': 2, '_id': '128', 'count': 519}
{'Top': 3, '_id': '062', 'count': 406}
{'Top': 4, '_id': '041', 'count': 399}
{'Top': 5, '_id': '004', 'count': 346}
{'Top': 6, '_id': '140', 'count': 345}
{'Top': 7, '_id': '017', 'count': 265}
{'Top': 8, '_id': '003', 'count': 261}
{'Top': 9, '_id': '014', 'count': 236}
{'Top': 10, '_id': '030', 'count': 210}
{'Top': 11, '_id': '011', 'count': 201}
{'Top': 12, '_id': '039', 'count': 198}
{'Top': 13, '_id': '034', 'count': 180}
{'Top': 14, '_id': '000', 'count': 155}
{'Top': 15, '_id': '002', 'count': 146}
{'Top': 16, '_id': '142', 'count': 138}
{'Top': 17, '_id': '037', 'count': 129}
{'Top': 18, '_id': '013', 'count': 119}
{'Top': 19, '_id': '042', 'count': 110}
{'Top': 20, '_id': '020', 'count': 94}


### Task 4
Find all users who have taken a taxi. 

In [63]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$match": {"transportation_mode" : "taxi"}},
        {"$group": {"_id":  "$user_id"}},
        {"$sort": {"_id": 1}},

    ]
)

for row in rows:
    print(row)

{'_id': '010'}
{'_id': '058'}
{'_id': '062'}
{'_id': '078'}
{'_id': '080'}
{'_id': '085'}
{'_id': '098'}
{'_id': '111'}
{'_id': '128'}
{'_id': '163'}


### Task 5
Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.

### Task 6
a) Find the year with the most activities.

b) Is this also the year with most recorded hours?

### Task 7
Find the total distance (in km) walked in 2008, by user with id=112.

### Task 8

 Find the top 20 users who have gained the most altitude meters

In [16]:
collection = connector.db["TrackPoint"]

rows = collection.aggregate(
    [
        {"$match": {"altitude": {"$ne": -777}}},
        {
            "$setWindowFields": {
                "partitionBy": "$activity_id",
                "sortBy": {"date_time": 1},
                "output": {
                    "previous_altitude": {"$shift": {"output": "$altitude", "by": -1}}
                },
            }
        },
        {
            "$project": {
                "activity_id": "$activity_id",
                "difference": {"$subtract": ["$altitude", "$previous_altitude"]},
            }
        },
        {"$match": {"difference": {"$gt": 0}}},
        {
            "$lookup": {
                "from": "Activity",
                "localField": "activity_id",
                "foreignField": "_id",
                "as": "activity",
            }
        },
        {
            "$group": {
                "_id": {
                    "user_id": "$activity.user_id",
                },
                "sum": {"$sum": "$difference"},
            }
        },
        {
            "$project": {
                "altitude_in_meters": {"$multiply": ["$sum", 0.304]},
            }
        },
        {"$sort": {"altitude_in_meters": -1}},
        {
            "$setWindowFields": {
                "sortBy": {"altitude_in_meters": -1},
                "output": {"Top": {"$rank": {}}},
            }
        },
        {"$limit": 20},
    ]
)

for row in rows:
    print(row)

{'_id': {'user_id': ['004']}, 'altitude_in_meters': 331164.832, 'Top': 1}
{'_id': {'user_id': ['041']}, 'altitude_in_meters': 240136.9264, 'Top': 2}
{'_id': {'user_id': ['003']}, 'altitude_in_meters': 233050.35199999998, 'Top': 3}
{'_id': {'user_id': ['030']}, 'altitude_in_meters': 175218.608, 'Top': 4}
{'_id': {'user_id': ['128']}, 'altitude_in_meters': 152583.6192, 'Top': 5}
{'_id': {'user_id': ['039']}, 'altitude_in_meters': 146318.544, 'Top': 6}
{'_id': {'user_id': ['000']}, 'altitude_in_meters': 121185.95199999999, 'Top': 7}
{'_id': {'user_id': ['002']}, 'altitude_in_meters': 114760.912, 'Top': 8}
{'_id': {'user_id': ['025']}, 'altitude_in_meters': 108872.06719999999, 'Top': 9}
{'_id': {'user_id': ['037']}, 'altitude_in_meters': 98974.13119999999, 'Top': 10}
{'_id': {'user_id': ['140']}, 'altitude_in_meters': 94597.35894173228, 'Top': 11}
{'_id': {'user_id': ['062']}, 'altitude_in_meters': 74565.8496, 'Top': 12}
{'_id': {'user_id': ['017']}, 'altitude_in_meters': 62417.09760000001

### Task 9
Find all users who have invalid activities, and the number of invalid activities per user

### Task 10
Find the users who have tracked an activity in the Forbidden City of Beijing.

### Task 11
Find all users who have registered transportation_mode and their most used transportation_mode. 

In [18]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$match": {"transportation_mode": {"$ne": None}}},
        {
            "$group": {
                "_id": {
                    "user_id": "$user_id",
                    "transportation_mode": "$transportation_mode",
                },
                "count": {"$sum": 1},
            }
        },
        {
            "$lookup": {
                "from": "User",
                "let": {
                    "a_id": "$_id",
                    "a_has_labels": "$has_labels",
                },
                "pipeline": [{"$match": {"$expr": {"$eq": ["$$a_id", "$user_id"]}}}],
                "as": "saves_transportation",
            }
        },
        {"$match": {"saves_transportation": {"$ne": {"$a_has_labels": False}}}},
        {"$sort": {"_id.user_id": 1, "count": -1}},
        {
            "$group": {
                "_id": "$_id.user_id",
                "most_used_transportation_mode": {"$first": "$_id.transportation_mode"},
            }
        },
        {"$sort": {"_id": 1}},
    ]
)

for row in rows:
    pprint(row)

{'_id': '010', 'most_used_transportation_mode': 'taxi'}
{'_id': '020', 'most_used_transportation_mode': 'bike'}
{'_id': '021', 'most_used_transportation_mode': 'walk'}
{'_id': '052', 'most_used_transportation_mode': 'bus'}
{'_id': '056', 'most_used_transportation_mode': 'bike'}
{'_id': '058', 'most_used_transportation_mode': 'car'}
{'_id': '060', 'most_used_transportation_mode': 'walk'}
{'_id': '062', 'most_used_transportation_mode': 'walk'}
{'_id': '064', 'most_used_transportation_mode': 'bike'}
{'_id': '065', 'most_used_transportation_mode': 'bike'}
{'_id': '067', 'most_used_transportation_mode': 'walk'}
{'_id': '069', 'most_used_transportation_mode': 'bike'}
{'_id': '073', 'most_used_transportation_mode': 'walk'}
{'_id': '075', 'most_used_transportation_mode': 'walk'}
{'_id': '076', 'most_used_transportation_mode': 'car'}
{'_id': '078', 'most_used_transportation_mode': 'walk'}
{'_id': '080', 'most_used_transportation_mode': 'taxi'}
{'_id': '081', 'most_used_transportation_mode': 'bi